In [ ]:
import transformer_lens.utils as utils
from transformer_lens import HookedTransformer, HookedTransformerConfig
import einops
import torch
from tqdm import tqdm
import numpy as np
from fancy_einsum import einsum
import chess
import numpy as np
import csv
from dataclasses import dataclass
from torch.nn import MSELoss, L1Loss
import pandas as pd
import pickle
import os
import logging

import chess_utils
import train_test_chess
from train_test_chess import Config, LinearProbeData

In [ ]:
torch.set_grad_enabled(False)
CAA_DIR = "contrastive_activations/"

In [ ]:
state_dict_name = f"{CAA_DIR}lichess_train_layer_12_pos_start_25_activations.pt"
state_dict = torch.load(state_dict_name, map_location=torch.device('cpu'))
print(state_dict.keys())
average_high_elo = state_dict["average_high_elo_activation"]
average_low_elo = state_dict["average_low_elo_activation"]
difference_vector = average_high_elo - average_low_elo
state_dict["difference_vector"] = difference_vector
torch.save(state_dict, state_dict_name)

state_dict2_name = f"{CAA_DIR}lichess_train_layer_12_pos_start_25_num_games_20000_activations.pt"
state_dict2 = torch.load(state_dict2_name, map_location=torch.device('cpu'))
print(state_dict2.keys())
average_high_elo2 = state_dict2["average_high_elo_activation"]
average_low_elo2 = state_dict2["average_low_elo_activation"]
difference_vector2 = average_high_elo2 - average_low_elo2
state_dict2["difference_vector"] = difference_vector2
torch.save(state_dict2, state_dict2_name)

In [ ]:
print(difference_vector.shape)
print(state_dict["layer"])
print(state_dict["pos_start"])
print(state_dict2["layer"])
print(state_dict2["pos_start"])

In [ ]:
from torch.nn.functional import cosine_similarity
# Calculating average values of each tensor
avg_value_high_elo = torch.mean(average_high_elo)
avg_value_low_elo = torch.mean(average_low_elo)
avg_value_difference = torch.mean(difference_vector)

avg_value_high_elo2 = torch.mean(average_high_elo2)
avg_value_low_elo2 = torch.mean(average_low_elo2)
avg_value_difference2 = torch.mean(difference_vector2)

# Calculating cosine similarity between all pairs
cos_sim_high_low = cosine_similarity(average_high_elo.unsqueeze(0), average_low_elo.unsqueeze(0)).item()
cos_sim_high_diff = cosine_similarity(average_high_elo.unsqueeze(0), difference_vector.unsqueeze(0)).item()
cos_sim_low_diff = cosine_similarity(average_low_elo.unsqueeze(0), difference_vector.unsqueeze(0)).item()

cos_sim_high_low2 = cosine_similarity(average_high_elo2.unsqueeze(0), average_low_elo2.unsqueeze(0)).item()
cos_sim_high_diff2 = cosine_similarity(average_high_elo2.unsqueeze(0), difference_vector2.unsqueeze(0)).item()
cos_sim_low_diff2 = cosine_similarity(average_low_elo2.unsqueeze(0), difference_vector2.unsqueeze(0)).item()

print(avg_value_high_elo, avg_value_low_elo, avg_value_difference, cos_sim_high_low, cos_sim_high_diff, cos_sim_low_diff)
print(avg_value_high_elo2, avg_value_low_elo2, avg_value_difference2, cos_sim_high_low2, cos_sim_high_diff2, cos_sim_low_diff2)


In [ ]:
cos_sim_high_high = cosine_similarity(average_high_elo.unsqueeze(0), average_high_elo2.unsqueeze(0)).item()
cos_sim_low_low = cosine_similarity(average_low_elo.unsqueeze(0), average_low_elo2.unsqueeze(0)).item()
cos_sim_diff_diff = cosine_similarity(difference_vector.unsqueeze(0), difference_vector2.unsqueeze(0)).item()

print(cos_sim_high_high, cos_sim_low_low, cos_sim_diff_diff)

cos_sim_high_low2 = cosine_similarity(average_high_elo.unsqueeze(0), average_low_elo2.unsqueeze(0)).item()
cos_sim_high_diff2 = cosine_similarity(average_high_elo.unsqueeze(0), difference_vector2.unsqueeze(0)).item()
cos_sim_low_diff2 = cosine_similarity(average_low_elo.unsqueeze(0), difference_vector2.unsqueeze(0)).item()
cos_sim_low_high2 = cosine_similarity(average_low_elo.unsqueeze(0), average_high_elo2.unsqueeze(0)).item()

print(cos_sim_high_low2, cos_sim_high_diff2, cos_sim_low_diff2, cos_sim_low_high2)